In [10]:
# Import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import representativeClustering as rc

In [11]:
# Load the data file
dataset = pd.read_csv("dataset", header=None, delimiter= ' ')

In [12]:
# dataset.head()

In [13]:
words = dataset[0]
features = dataset.drop(0, 'columns')
features = features.to_numpy()

In [14]:
# def distance(x, y):
#     return np.linalg.norm(x - y)

In [15]:
# def assign(features, clusterRepresentatives):

#     '''
#     For each feature calculate distance from each cluster representative.
#     Assign this feature the representative which is closest to it
#     '''
#     # The index numbers of clusters array corresponds to the object
#     # at same index number in the feature matrix
#     # This array shall contain information about the cluster 
#     # assigned to the corresponding object in the feature matrix
#     # clusters = np.zeros((len(features), 1))
#     clusters = np.zeros((len(features)))

#     for fIndex in range(len(features)):

#         # Calculate distance of this feature from each cluster representative
#         clusterDistance = [distance(features[fIndex], c) for c in clusterRepresentatives]

#         # Find the index number corresponding to the minimum distance
#         # This index number also corresponds to the closest cluster representative
#         # Thus, the array 'clusters' contain the index number of the cluster 
#         # representative which is closest to this object
#         clusters[fIndex] = np.argmin(clusterDistance)

        
#     return clusters

In [16]:
# def optimise(features, clusters, clusterRepresentatives):

#     '''
#     For each cluster, find mean of features of all the object
#     which are assigned to it. This mean shall be designated
#     as the new cluster representative for that cluster
#     Return the array of the new cluster representatives
#     '''

#     for cIndex in range(len(clusterRepresentatives)):
        
#         # Find the objects which are assigned to the current cluster
#         # This matrix contain feature values of the objects in the
#         # current cluster
#         objectsInThisCluster = features[clusters == cIndex]

#         # Find row-vise mean of all these objects which are assigned
#         # to the current cluster
#         # This gives, for each feature, mean over all the objects in this cluster
#         clusterRepresentatives[cIndex] = np.mean(objectsInThisCluster, axis = 0)
    

#     return clusterRepresentatives

In [17]:
# def k_means(features, k):

#     # Initialisation Phase
#     np.random.seed(42)
#     initialRandomObjects = np.random.choice(range(features.shape[0]), size=k)

#     # Choose initial cluster representatives as randomly chosen objects from the dataset
#     clusterRepresentatives = features[initialRandomObjects]
    

#     # Assignment Phase
#     # Assign cluster representative to each object (i.e., each row of features)
#     clusters = assign(features, clusterRepresentatives)
    
#     # Optimisation Phase
#     # Compute new representative as mean of the current clusters
#     clusterRepresentatives = optimise(features, clusters, clusterRepresentatives)
    
    

In [18]:
# rc.k_means(features, 4)

(array([2., 2., 0., 2., 1., 2., 2., 3., 2., 3., 2., 2., 2., 2., 2., 3., 2.,
        2., 3., 2., 3., 2., 2., 2., 2., 1., 1., 1., 2., 3., 1., 1., 2., 2.,
        2., 3., 1., 2., 2., 1., 2., 2., 2., 1., 2., 1., 2., 2., 3., 3., 2.,
        2., 2., 2., 2., 3., 2., 2., 3., 0., 2., 2., 2., 2., 2., 0., 2., 2.,
        2., 2., 2., 3., 2., 2., 2., 2., 2., 2., 3., 2., 2., 0., 2., 2., 0.,
        2., 2., 2., 2., 0., 2., 2., 2., 2., 2., 0., 2., 2., 0., 0., 2., 0.,
        0., 2., 0., 2., 2., 2., 2., 2., 2., 2., 0., 2., 3., 3., 0., 2., 0.,
        0., 0., 2., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 0.,
        0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 2.,
        2., 2., 2., 2., 2., 2., 0., 2., 2., 3., 0., 2., 2., 2., 2., 2., 2.,
        2., 0., 3., 3., 2., 0., 2., 2., 2., 0., 2., 3., 0., 2., 0., 2., 2.,
        2., 0., 0., 2., 2., 2., 2., 2., 2., 2., 2., 0., 2., 2., 2., 3., 2.,
        2., 2., 2., 2., 2., 2., 2., 3., 1., 1., 3., 1., 3., 1., 1., 1., 1.,
        3., 